In [1]:
import sys
import os

working_dir = os.path.expanduser('~/prediction-validation')
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
import pandas as pd
import numpy as np
import torch
import pickle
from argparse import Namespace

from utils.utilities_DL import get_loss,load_model_and_optimizer
from build_inputs.load_datasets_to_predict import load_datasets_to_predict
from examples.load_best_config import get_trainer_and_ds_from_saved_trial,load_trainer_ds_from_saved_trial

def get_TS_prediction(trainer,ds,stations_to_plot=['CHA'],training_mode='test',name= ''):
    spatial_units  = list(ds.spatial_unit)
    if stations_to_plot is None:
        stations_to_plot = spatial_units
    station_indices = [spatial_units.index(station) for station in stations_to_plot]
    predict,Y_true,_ = trainer.testing(ds.normalizer, training_mode =training_mode)
    df_pred =  pd.DataFrame({f'{name}_{spatial_units[station_i]}_Pred':predict[:,station_i,0].detach().cpu().numpy() for station_i in station_indices})
    df_true = pd.DataFrame({f'{spatial_units[station_i]}_Y_true': Y_true[:,station_i,0].detach().cpu().numpy() for station_i in station_indices})
    index_df = getattr(ds.tensor_limits_keeper,f"df_verif_{training_mode}").iloc[:,-1].values
    return df_pred,df_true,index_df


if False:
    working_dir = os.path.expanduser('~/prediction-validation/save')
    subfolder = 'best_models'
    trial_id ='PeMS08_flow_PeMS08_speed_PeMS08_occupancy_calendar_STAEformer_HuberLossLoss_2025_05_22_16_06_17612'
    trial_id ='PeMS08_flow_PeMS08_speed_PeMS08_occupancy_calendar_STAEformer_HuberLossLoss_2025_05_22_16_06_17612'
    # 'subway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371'
    # 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229'
    # 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229'
    # 'subway_in_calendar_STGCN_MSELoss_2024_08_25_22_56_92429'
    # 'netmob_subway_in_STGCN_ImageAvgPooling_MSELoss_2024_08_24_01_42_17375'
    # 'netmob_subway_in_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_23_06_53_46982'
    # 'netmob_subway_in_calendar_STGCN_ImageAvgPooling_MSELoss_2024_08_27_00_16_90667'
    # 'netmob_subway_in_calendar_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_28_06_04_41108'
    # 'subway_in_STGCN_MSELoss_2024_08_21_14_50_2810'
    model_id = f"{trial_id}_F6_f0"     # One trial_id can be associated to several folds 

if True:
    working_dir = os.path.expanduser('~/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/optim')
    # subfolder = 'STGCN_architecture'
    subfolder = 'best_models'
    trial_id = '_calendar'
    trial_id = '_calendar_Google_Maps_Deezer_IRIS_clustering015'
    trial_id = '_Init'

    # subfolder = 'CRITER_3_4_5_lanes_flow_STAEformer'
    model_id = f"{trial_id}_f5"     # One trial_id can be associated to several folds 


model_save_path = f"{working_dir}/{subfolder}/{model_id}.pkl"

print('Model path: ',model_save_path)

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training and Hyper-parameter tuning with Ray is not possible
Model path:  /home/rrochas/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/optim/best_models/_Init_f5.pkl


In [2]:
model_args = pickle.load(open(f"{working_dir}/{subfolder}/model_args.pkl",'rb'))
args = model_args['model'][model_id]['args']
args = Namespace(**args)


test_metrics = model_args['model'][model_id]['performance']['test_metrics']

print(f"\n--------- Test ---------\nAll Steps RMSE = {test_metrics['rmse_all']}, MAE = {test_metrics['mae_all']}, MAPE = {test_metrics['mape_all']}")
for h in np.arange(1,args.step_ahead+1):
    print(f"Step {h} RMSE = {test_metrics[f'rmse_h{h}']}, MAE = {test_metrics[f'mae_h{h}']}, MAPE = {test_metrics[f'mape_h{h}']}")


--------- Test ---------
All Steps RMSE = 52.836530685424805, MAE = 37.24277591705322, MAPE = 7.8500648736953735
Step 1 RMSE = 40.00498580932617, MAE = 28.864337921142578, MAPE = 6.60634708404541
Step 2 RMSE = 49.3797721862793, MAE = 35.085243225097656, MAPE = 7.536723613739014
Step 3 RMSE = 57.39794921875, MAE = 40.200626373291016, MAPE = 8.26611614227295
Step 4 RMSE = 64.56341552734375, MAE = 44.82089614868164, MAPE = 8.991072654724121


## Spatial Agg Velo'v

In [1]:
data_path = "/home/rrochas/../../../data/rrochas/prediction_validation/"

iris_path = f"{data_path}/lyon_iris_shapefile/lyon.shp"
velov_json = f"{data_path}/pvo_patrimoine_voirie.pvostationvelov.json"

# Import json with gpd: 
import geopandas as gpd 
velov_gdf = gpd.read_file(velov_json)
iris_gdf = gpd.read_file(iris_path)

display(velov_gdf.head(2))
display(iris_gdf.head(2))

ERROR 1: PROJ: proj_create_from_database: Open of /root/anaconda3/envs/pytorch-2.0.1/share/proj failed


,idstation,nom,adresse1,adresse2,commune,numdansarrondissement,nbbornettes,stationbonus,pole,ouverte,achevement,gid,code_insee,geometry
0,7024,Bancel / Chevreul,Rue Bancel,Rue Chevreul,Lyon 7 ème,24.0,21.0,,Itinéraire cyclable,Oui,,1,69387,POINT (4.84052 45.74852)
1,10063,Perrin / Jean Jaurès,"9, Place Jules Grandclément",Rue Antonin Perrin,VILLEURBANNE,63.0,34.0,,quartier Grandclément,Oui,,2,69266,POINT (4.88561 45.75839)


,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,x,y,NEIGHBORS,STATION,geometry
0,69286.0,Rillieux-la-Pape,202.0,692860202.0,Mont Blanc,H,4.895028,45.810221,"692860402,692860102,692860301,692860201",None,"POLYGON ((4.89204 45.81283, 4.89271 45.81305, ..."
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,"POLYGON ((4.82108 45.75066, 4.82109 45.75066, ..."


In [2]:
import sys
import os
import pandas as pd
import torch
import numpy as np
from datetime import datetime

# --- Gestion de l'arborescence ---
current_file_path = os.path.abspath(os.path.dirname(os.getcwd()))
parent_dir = os.path.abspath(os.path.join(current_file_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# --- Importations personnalisées ---
from dataset import DataSet, PersonnalInput
from utils.utilities import filter_args # Assurez-vous que ce chemin est correct
from build_inputs.load_preprocessed_dataset import load_input_and_preprocess
# --- Constantes spécifiques à cette donnée ---
FOLDER_PATH = os.path.expanduser('~/../../../data/rrochas/prediction_validation') # Chemin vers le dossier contenant les données
NAME = 'bike_in'
FILE_BASE_NAME = 'velov'
DIRECTION = 'attracted' #'attracted' # emitted
FILE_PATTERN = f'velov_{DIRECTION}_by_station' # Sera complété par args.freq
DATA_SUBFOLDER = 'agg_data/velov' # Sous-dossier dans FOLDER_PATH


# Couverture théorique
START = '2019-01-01' 
END = '2020-01-01'
USELESS_DATES = {'hour':[], #[1,2,3,4,5,6],  #[] if no useless (i.e removed) hours
                 'weekday':[]#[5,6],
                 }
# Liste des périodes invalides
list_of_invalid_period = []

C = 1 # Nombre de canaux/features par unité spatiale

# Colonnes attendues dans le CSV
if DIRECTION == 'emitted':
    DATE_COL = 'date_sortie' 
    LOCATION_COL = 'id_sortie' # Ou 'id_entree' pour 'attracted'?
    VALUE_COL = 'volume'
else:
    DATE_COL = 'date_retour' 
    LOCATION_COL = 'id_retour' # Ou 'id_entree' pour 'attracted'?
    VALUE_COL = 'volume'

target_freq = '15min'
# Construction spécifique du nom de fichier pour velov
file_name = f"{FILE_PATTERN}{target_freq}"
data_file_path = os.path.join(FOLDER_PATH, DATA_SUBFOLDER, f"{file_name}.csv")

print(f"Chargement des données depuis : {data_file_path}")
try:
    df = pd.read_csv(data_file_path)
except FileNotFoundError:
    print(f"ERREUR : Le fichier {data_file_path} n'a pas été trouvé.")
    print(f"Vérifiez que la fréquence '{target_freq}' existe pour velov_{DIRECTION} et que les chemins sont corrects.")
except Exception as e:
    print(f"ERREUR lors du chargement du fichier {file_name}.csv: {e}")

Chargement des données depuis : /home/rrochas/../../../data/rrochas/prediction_validation/agg_data/velov/velov_attracted_by_station15min.csv


In [44]:
def compute_score(row,gdf):
    # Init score:
    best_score = float('inf')
    best_neighbor = None
    
    # Compute: 
    area_i = row['area']
    geom_i = row.geometry

    for iris_code in row['NEIGHBORS']:
        row_j = gdf.loc[iris_code]
        common = geom_i.union(row_j.geometry)
        perim = common.length/1e3  # Convertir en km
        area = area_i + row_j['area']
        score = (area) / perim

        if score < best_score:
            best_score = score
            best_neighbor = iris_code

    return best_score,best_neighbor
        
def iteration_spatial_agg(gdf):
    ## Find index of the best score (i.e min score):
    best_score_index = list(gdf['best_score'].nsmallest(2).index)
    aggregated_rows = gdf.loc[best_score_index].copy()
    ## Merge 2 rows: 
    row1 = gdf.loc[best_score_index[0]]
    row2 = gdf.loc[best_score_index[1]]
    row = row1.copy()

    # Update Neighbors, Station & Geometry:
    row.NEIGHBORS = list((set(row1.NEIGHBORS) | set(row2.NEIGHBORS)) - set([row1.name,row2.name]))
    row.STATION = list((set(row1.STATION) | set(row2.STATION)))
    row.geometry = row1.geometry.union(row2.geometry)
    # row.geometry = unary_union([row_i.geometry, row_j.geometry]).convex_hull
    row['area'] = row.geometry.area /1e6


    ## Remove row2 and row1 from gdf: 
    gdf.drop(index=best_score_index[0], inplace=True)
    gdf.drop(index=best_score_index[1], inplace=True)

    ## Add row to gdf:
    gdf.loc[row.name] = row

    # print('row1.name: ', row1.name)
    # print('row2.name: ', row2.name)
    # print('All neighbords: ', row2.NEIGHBORS)

    ## Change all name of neighbor2 to the name of neighbor1  
    gdf_neighbords = gdf.copy()
    for neighbor in row2.NEIGHBORS:
        # If neighbor is not in row1.NEIGHBORS, add it:
        if neighbor != row1.name : 
            # print('\nneighbor: ', neighbor)
            # print("gdf.loc[neighbor, 'NEIGHBORS'] before: ", gdf.loc[neighbor, 'NEIGHBORS'])
            new_neighbors = list(set(gdf_neighbords.loc[neighbor]['NEIGHBORS'] + [row1.name]) - set([row2.name]) )
            gdf.at[neighbor, 'NEIGHBORS'] = new_neighbors
            # print("gdf.loc[neighbor, 'NEIGHBORS'] after: ", gdf.loc[neighbor]['NEIGHBORS'] )

    ## Compute New Scores: 
    row['best_score, best_neighbor'] = compute_score(row, gdf)

    ## Compute new best_score and best_neighbor of each neighbor :
    for neighbor in row.NEIGHBORS:
        best_score, best_neighbor = compute_score(gdf.loc[neighbor], gdf)
        # gdf.loc[neighbor, 'best_score'] = best_score
        # gdf.loc[neighbor, 'best_neighbor'] = best_neighbor
        gdf.at[neighbor, 'best_score']= best_score
        gdf.at[neighbor, 'best_neighbor']= best_neighbor

    gdf.crs = 'EPSG:2154'
    return gdf,aggregated_rows


n_target = 50
gdf = iris_gdf.copy()

gdf.crs = 'EPSG:4326' 
gdf = gdf.to_crs('EPSG:2154')  # Projection Lambert-93
# Conversion des types
gdf['INSEE_COM'] = gdf['INSEE_COM'].astype(int)
gdf['IRIS'] = gdf['IRIS'].astype(int)
gdf['CODE_IRIS'] = gdf['CODE_IRIS'].astype(int)
# Parsing de la colonne NEIGHBORS
gdf['NEIGHBORS'] = gdf['NEIGHBORS'].apply(
    lambda s: list(map(int, s.split(','))) if isinstance(s, str) and s else []
)
gdf['STATION'] = gdf['STATION'].apply(
    lambda s: list(map(int, s.split(' '))) if isinstance(s, str) and s else []
)
gdf['area'] = gdf.geometry.area/ 1e6  # Convertir en km²
gdf.set_index('CODE_IRIS', inplace=True)
gdf[['best_score','best_neighbor']] = pd.DataFrame(gdf.apply(lambda row: compute_score(row,gdf),axis=1).tolist(),columns=['best_score','best_neighbor'],index=gdf.index)

display(gdf.head(2))

# Boucle principale
init_len_gdf = len(gdf)
k  = 0
while len(gdf) > n_target:
    gdf,aggregated_rows = iteration_spatial_agg(gdf)
    if k== 5:
        break

    k +=1
gdf['NEIGHBORS'] = gdf['NEIGHBORS'].apply(lambda x: ','.join(map(str, x)))
gdf['STATION'] = gdf['STATION'].apply(lambda x: ' '.join(map(str, x)))


aggregated_rows['NEIGHBORS'] = aggregated_rows['NEIGHBORS'].apply(lambda x: ','.join(map(str, x)))
aggregated_rows['STATION'] = aggregated_rows['STATION'].apply(lambda x: ' '.join(map(str, x)))
aggregated_rows.explore()

,INSEE_COM,NOM_COM,IRIS,NOM_IRIS,TYP_IRIS,x,y,NEIGHBORS,STATION,geometry,area,best_score,best_neighbor
CODE_IRIS,,,,,,,,,,,,,
692860202,69286,Rillieux-la-Pape,202,Mont Blanc,H,4.895028,45.810221,"[692860402, 692860102, 692860301, 692860201]",[],"POLYGON ((846914.700 6525449.300, 846965.600 6...",0.322462,0.122545,692860402
693820501,69382,Lyon 2e Arrondissement,501,Montrochet-Marché-Gare,A,4.818996,45.738332,"[693870103, 693850104, 693820503, 691420102, 6...","[2020, 2005, 2009, 2019]","POLYGON ((841565.100 6518417.300, 841565.300 6...",1.124361,0.122897,693870103


In [40]:
gdf.explore()

In [34]:
aggregated_rows.explore()

In [27]:
gdf.crs = 'epsg:2154'
#gdf.to_crs('epsg:4326')
gdf['geometry'].explore()

In [4]:
best_score_index = list(gdf['best_score'].nsmallest(2).index)

## Merge 2 rows: 
row1 = gdf.loc[best_score_index[0]]
row2 = gdf.loc[best_score_index[1]]
row = row1.copy()

# Update Neighbors, Station & Geometry:
row.NEIGHBORS = list((set(row1.NEIGHBORS) | set(row2.NEIGHBORS)) - set([row1.name,row2.name]))
row.STATION = list((set(row1.STATION) | set(row2.STATION)))
row.geometry = row1.geometry.union(row2.geometry)
# row.geometry = unary_union([row_i.geometry, row_j.geometry]).convex_hull
row['area'] = row.geometry.area / 1e6


## Remove row2 and row1 from gdf: 
gdf.drop(index=best_score_index[0], inplace=True)
gdf.drop(index=best_score_index[1], inplace=True)

## Add row to gdf:
gdf.loc[row.name] = row

print('row1.name: ', row1.name)
print('row2.name: ', row2.name)

# Update the neighbors of the neighbor to include row1 and remove row2
gdf_neighbords = gdf.copy()
for neighbor in row2.NEIGHBORS:
    # If neighbor is not in row1.NEIGHBORS, add it:
    if neighbor != row1.name : 
        print('\nneighbor: ', neighbor)
        print("gdf.loc[neighbor, 'NEIGHBORS'] before: ", gdf.loc[neighbor, 'NEIGHBORS'])
        gdf.loc[neighbor]['NEIGHBORS'] = list(set(gdf_neighbords.loc[neighbor]['NEIGHBORS'] + [row1.name]) - set([row2.name]) )
        print("gdf.loc[neighbor, 'NEIGHBORS'] after: ", gdf.loc[neighbor, 'NEIGHBORS'])

row1.name:  693810202
row2.name:  693820103

neighbor:  693820202
gdf.loc[neighbor, 'NEIGHBORS'] before:  [693850104, 693820301, 693820201, 693820103, 693820204, 693820102, 693850102]
gdf.loc[neighbor, 'NEIGHBORS'] after:  [693850104, 693820301, 693820201, 693820103, 693820204, 693820102, 693850102]

neighbor:  693820102
gdf.loc[neighbor, 'NEIGHBORS'] before:  [693850104, 693860401, 693830105, 693820201, 693820103, 693830102, 693820202, 693820101, 693850102]
gdf.loc[neighbor, 'NEIGHBORS'] after:  [693850104, 693860401, 693830105, 693820201, 693820103, 693830102, 693820202, 693820101, 693850102]


/tmp/ipykernel_175479/2937949293.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.loc[neighbor]['NEIGHBORS'] = list(set(gdf_neighbords.loc[neighbor]['NEIGHBORS'] + [row1.name]) - set([row2.name]) )


In [208]:
gdf['geometry'].explore()

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

def aggregate_iris_zones(gdf: gpd.GeoDataFrame, n_target: int) -> gpd.GeoDataFrame:
    """
    Agrège les zones IRIS d'un GeoDataFrame jusqu'à atteindre au plus n_target zones.
    L'agrégation minimise le critère (s_i + s_j) / P_ij entre zones adjacentes.

    Paramètres:
        gdf: GeoDataFrame avec colonnes 'INSEE_COM', 'IRIS', 'CODE_IRIS', 'NEIGHBORS', 'STATION', 'geometry', etc.
        n_target: nombre cible de zones après agrégation.

    Retour:
        GeoDataFrame agrégé.
    """
    # Copie pour ne pas modifier l'original
    gdf = gdf.copy()
    gdf.crs = 'EPSG:4326' 
    gdf = gdf.to_crs('EPSG:2154')  # Projection Lambert-93
    # Conversion des types
    gdf['INSEE_COM'] = gdf['INSEE_COM'].astype(int)
    gdf['IRIS'] = gdf['IRIS'].astype(int)
    gdf['CODE_IRIS'] = gdf['CODE_IRIS'].astype(int)
    # Parsing de la colonne NEIGHBORS
    gdf['NEIGHBORS'] = gdf['NEIGHBORS'].apply(
        lambda s: list(map(int, s.split(','))) if isinstance(s, str) and s else []
    )

    # Boucle principale
    while len(gdf) > n_target:
        # Calcul des surfaces
        gdf['area'] = gdf.geometry.area/ 1e6  # Convertir en km²
        # Dictionnaire code -> index
        code_to_idx = dict(zip(gdf['CODE_IRIS'], gdf.index))
        # Calcul des scores temporaires
        tmp_index = {}
        tmp_score = {}
        for idx, row in gdf.iterrows():
            area_i = row['area']
            geom_i = row.geometry
            best_score = float('inf')
            best_neighbor = None
            for j_code in row['NEIGHBORS']:
                if j_code not in code_to_idx:
                    continue
                j_idx = code_to_idx[j_code]
                row_j = gdf.loc[j_idx]
                common = geom_i.intersection(row_j.geometry)
                perim = common.length
                if perim <= 0:
                    continue
                score = (area_i + row_j['area']) / perim
                if score < best_score:
                    best_score = score
                    best_neighbor = j_idx
            tmp_index[idx] = best_neighbor
            tmp_score[idx] = best_score
        gdf['tmp_index_fusion'] = pd.Series(tmp_index)
        gdf['tmp_score'] = pd.Series(tmp_score)
        # Sélection des paires valides
        valid = gdf[gdf['tmp_index_fusion'].notnull()]
        if valid.empty:
            break
        # Choix de la fusion à effectuer
        i_idx = valid['tmp_score'].idxmin()
        j_idx = int(valid.loc[i_idx, 'tmp_index_fusion'])
        row_i = gdf.loc[i_idx].copy()
        row_j = gdf.loc[j_idx].copy()
        # Géométrie agrégée
        new_geom = unary_union([row_i.geometry, row_j.geometry]).convex_hull
        # Neighbors fusionnés
        neigh_i = set(row_i['NEIGHBORS'])
        neigh_j = set(row_j['NEIGHBORS'])
        new_neighbors = list((neigh_i | neigh_j) - {row_i['CODE_IRIS'], row_j['CODE_IRIS']})
        # Stations fusionnées
        def parse_stations(x):
            if pd.isna(x):
                return []
            return x.split() if isinstance(x, str) else [x]
        stations = set(parse_stations(row_i['STATION']) + parse_stations(row_j['STATION']))
        new_station = ' '.join(stations) if stations else None
        # Mise à jour de la zone i
        gdf.at[i_idx, 'geometry'] = new_geom
        gdf.at[i_idx, 'NEIGHBORS'] = new_neighbors
        gdf.at[i_idx, 'STATION'] = new_station
        # Remplacement du code j par i chez les voisins
        code_i = row_i['CODE_IRIS']
        code_j = row_j['CODE_IRIS']
        for idx_k, row_k in gdf.iterrows():
            kn = row_k['NEIGHBORS']
            if code_j in kn:
                updated = list(set([code_i if x == code_j else x for x in kn]))
                # Enlever auto-références
                if row_k['CODE_IRIS'] in updated:
                    updated.remove(row_k['CODE_IRIS'])
                gdf.at[idx_k, 'NEIGHBORS'] = updated
        # Suppression de la zone j
        gdf = gdf.drop(index=j_idx)
        # Nettoyage des colonnes temporaires
        gdf = gdf.drop(columns=['area', 'tmp_index_fusion', 'tmp_score'], errors=True)
    return gdf


target_n = 50

agg_gdf = aggregate_iris_zones(iris_gdf, target_n)
agg_gdf.explore()

KeyError: "['area', 'tmp_index_fusion', 'tmp_score'] not found in axis"

In [106]:
iris_gdf

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,x,y,NEIGHBORS,STATION,geometry
0,69286.0,Rillieux-la-Pape,202.0,692860202.0,Mont Blanc,H,4.895028,45.810221,"692860402,692860102,692860301,692860201",None,"POLYGON ((4.89204 45.81283, 4.89271 45.81305, ..."
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,"POLYGON ((4.82108 45.75066, 4.82109 45.75066, ..."
2,69275.0,Décines-Charpieu,107.0,692750107.0,Bonneveau,H,4.960622,45.762751,"692750109,692750108,692750106,692750101",None,"POLYGON ((4.96062 45.76006, 4.96035 45.76008, ..."
3,69386.0,Lyon 6e Arrondissement,201.0,693860201.0,Puvis de Chavannes,H,4.842614,45.771162,"693860302,693860104,693860202,693810101,693860...",6012,"POLYGON ((4.84567 45.77084, 4.84465 45.77073, ..."
4,69149.0,Oullins,303.0,691490303.0,Montmein,H,4.808261,45.708957,"691490302,691490501,691490502,691490401,691520...",None,"POLYGON ((4.80838 45.71222, 4.80848 45.71214, ..."
...,...,...,...,...,...,...,...,...,...,...,...
425,69266.0,Villeurbanne,502.0,692660502.0,Croix Luizet Est,H,4.880359,45.779421,"692660501,692660702,692660601,692660201,692660901",10008,"POLYGON ((4.88381 45.77835, 4.88302 45.77817, ..."
426,69384.0,Lyon 4e Arrondissement,501.0,693840501.0,Lyon Plage-Ypres,H,4.817217,45.783245,"693840402,693890201,693840401,690340101,690340...",4042 4024,"POLYGON ((4.81901 45.78979, 4.81977 45.78930, ..."
427,69381.0,Lyon 1er Arrondissement,302.0,693810302.0,Trois Gaules,H,4.829512,45.770813,"693810501,693810304,693810303,693810301,693810...",1034,"POLYGON ((4.82572 45.77040, 4.82573 45.77085, ..."
428,69383.0,Lyon 3e Arrondissement,501.0,693830501.0,Villette-Paul Bert,H,4.863821,45.756227,"693830301,693830502,693830602,693830302,693830...",3099 3066,"POLYGON ((4.86287 45.75860, 4.86337 45.75854, ..."


In [97]:
iris_gdf.sjoin(velov_gdf)

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,x,y,NEIGHBORS,STATION,...,adresse2,commune,numdansarrondissement,nbbornettes,stationbonus,pole,ouverte,achevement,gid,code_insee
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,...,Parvis du Musée des Confluences,Lyon 2 ème,20.0,20.0,,Musée des Confluences,Oui,,217,69382
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,...,,Lyon 2 ème,NaN,20.0,,Lyon 2ème arrondissement,Oui,,415,69382
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,...,Rue Hrant Dink,Lyon 2 ème,9.0,26.0,,Confluence - Les Docks,Oui,Monté,288,69382
1,69382.0,Lyon 2e Arrondissement,501.0,693820501.0,Montrochet-Marché-Gare,A,4.818996,45.738332,"693870103,693850104,693820503,691420102,693820...",2020 2005 2009 2019,...,Esplanade François Miterrand,Lyon 2 ème,5.0,32.0,,Hôtel de Région - Tram Montrochet - Pôle de lo...,Oui,,270,69382
3,69386.0,Lyon 6e Arrondissement,201.0,693860201.0,Puvis de Chavannes,H,4.842614,45.771162,"693860302,693860104,693860202,693810101,693860...",6012,...,Angle rue Vendôme,Lyon 6 ème,12.0,10.0,,,Oui,,75,69386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,69384.0,Lyon 4e Arrondissement,501.0,693840501.0,Lyon Plage-Ypres,H,4.817217,45.783245,"693840402,693890201,693840401,690340101,690340...",4042 4024,...,,Lyon 4 ème,42.0,16.0,,Stade,Oui,,165,69384
427,69381.0,Lyon 1er Arrondissement,302.0,693810302.0,Trois Gaules,H,4.829512,45.770813,"693810501,693810304,693810303,693810301,693810...",1034,...,,Lyon 1 er,34.0,16.0,,,Oui,,44,69381
428,69383.0,Lyon 3e Arrondissement,501.0,693830501.0,Villette-Paul Bert,H,4.863821,45.756227,"693830301,693830502,693830602,693830302,693830...",3099 3066,...,,Lyon 3 ème,66.0,19.0,,"Bureaux, habitat, collège",Oui,,394,69383
428,69383.0,Lyon 3e Arrondissement,501.0,693830501.0,Villette-Paul Bert,H,4.863821,45.756227,"693830301,693830502,693830602,693830302,693830...",3099 3066,...,,Lyon 3 ème,99.0,18.0,,"Coeur de quartier, commerces",Oui,,174,69383


In [ ]:

# --- Prétraitement ---
try:
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])
    df_pivoted = df.pivot_table(index=DATE_COL, columns=LOCATION_COL, values=VALUE_COL, aggfunc='sum',fill_value = 0)

    # Reindex 
    df_pivoted.reindex(pd.date_range(start=START, end=END, freq=target_freq), fill_value=0, inplace=True)
    df_filtered = df_pivoted[df_pivoted.index.isin(coverage_period)].copy()

    local_df_dates = pd.DataFrame(df_filtered.index, columns=['date'])

    if df_filtered.empty:
        print(f"ERROR : No data available for {file_name}.csv in the specified coverage period.")
        return None

    # Convert into Tennsor
    data_T = torch.tensor(df_filtered.values).float()

except :
    print(f"ERROR during preprocessing of {file_name}.csv: DataFrame might be empty or invalid.")
    return None


dims = [0] # if [0] then Normalisation on temporal dim
processed_input = load_input_and_preprocess(dims = dims,normalize=normalize,invalid_dates=invalid_dates,args=args,data_T=data_T,coverage_period=coverage_period,name=name,
                                            minmaxnorm=minmaxnorm,standardize=standardize)

# --- Finalisation Métadonnées ---
processed_input.spatial_unit = df_filtered.columns.tolist()
processed_input.C = C
processed_input.periods = None # Pas de périodicité spécifique définie ici

print(f"Chargement et prétraitement de {file_base_name} terminés.")
return processed_input

# --- Point d'entrée pour exécution directe (optionnel, pour tests) ---
# ... (Similaire à subway_indiv.py, adapter les mocks) ...

In [3]:
stations_to_plot = [292,64,66,1000] #[893,67,999,65,291,1176]
modification = {'shuffle':False,
                'data_augmentation':False }
# trainer,ds,args = get_trainer_and_ds_from_saved_trial(args,model_save_path,modification=modification)
trainer, ds, args_init = load_trainer_ds_from_saved_trial(args,model_save_path,modification=modification)

----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 18480 elts between 2019-03-16 00:00:00 and 2019-05-31 23:54:00
Invalid dates within this fold: 791

>>>Tackle Target dataset: CRITER_3_4_5_lanes_flow
Number of invalid time-slots (i.e data when every single sensors does not have data): 82
number of nan values before filling :  539557
nb sparse_columns :  96
Number of sensors after filter sparse sensor :  51

Total anomalies detected: 4561
 Data loaded with shape: (18480, 51)
   Init Dataset: 'torch.Size([18480, 51]). 0 Nan values
   TRAIN contextual_ds: torch.Size([10558, 51, 10])
   VALID contextual_ds: torch.Size([3520, 51, 10])
   TEST contextual_ds: torch.Size([3519, 51, 10])
Init U/Utarget size: torch.Size([17598, 51, 10])/torch.Size([17598, 51, 4]) Train/Valid/Test 10558 3520 3519

----------------------------------------
Loading the dataset for fold n°5
Model size: 0.003GB


In [8]:
from plotting.TS_analysis import plot_TS


stations_to_plot = [893,67,999,1176] #65,291,1176] # [292,64,66,1000] #[893,67,999,65,291,1176]
df_pred1,df_true,index_df = get_TS_prediction(trainer,ds,stations_to_plot=stations_to_plot,training_mode='test',name= 'CRITER')
df_to_plot= pd.concat([df_pred1,df_true],axis=1)
df_to_plot.index = index_df
plot_TS(df_to_plot,width=1200,height=400,bool_show=True,title='Comparison of Prediction Quality')

Loading BokehJS ...

figure(id='p1243', ...)

In [ ]:
from calendar_class import get_time_slots_labels
from constants.paths import FILE_NAME
model_param = torch.load(model_save_path)

# Load Model:
coverage_period = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])
dataset,_,_ = load_datasets_to_predict(args,coverage_period)
_,dic_class2rpz,_,_ = get_time_slots_labels(dataset,nb_class = [0,1,2,3])
loss_function = get_loss(args)
model,optimizer,scheduler = load_model_and_optimizer(args,dic_class2rpz)

# Associate its weights: 
model.load_state_dict(model_param['state_dict'])

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

Init Subway-In Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 


 Tackling Training Set

 Tackling Validation Set

 Tackling Training Set

 Tackling Validation Set

U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([5145, 40, 7]) Utarget_train size:  torch.Size([5145, 40, 1])
U_valid size:  torch.Size([1477, 40, 7]) Utarget_valid size:  torch.Size([1477, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(1.)
U_valid min:  tensor(0.) U_valid max:  tensor(2.3191)
model size: 0.001GB
number of total parameters: 249889
number of trainable parameters: 249889


<All keys matched successfully>

In [ ]:
stations_to_plot = [292,64,66,1000] #[893,67,999,65,291,1176]
modification = {'shuffle':False,
                'data_augmentation':False }
save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
trial_id = 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
add_name_id = 'RE_CRITER_3lanes_rich_interpolation'
trainer,ds,args = get_trainer_and_ds_from_saved_trial(trial_id,add_name_id,save_folder,modification)

Training and Hyper-parameter tuning with Ray is not possible
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 387

total remaining idptm : 67
after filtering df: (3685, 67)
after removing NaN: (3685, 36)


NameError: name 'blabla' is not defined

In [2]:
trial_id = 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836'
add_name_id = 'RE_CRITER_3lanes_netmob_POIS_rich_interpolation_Waze_DL'
trainer2,ds2,args2 = get_trainer_and_ds_from_saved_trial(trial_id,add_name_id,save_folder,modification)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 387



Init Dataset: 'torch.Size([3685, 20]) with 73700 Total nb of elements and 0 Nan values
nb CRITER_3lanes invalid dates:  387
vision_input_type POIs
vision_model_name VariableSelectionNetwork
Init U/Utarget size: torch.Size([2344, 20, 7])/torch.Size([2344, 20, 1]) Train/Valid/Test 1406 469 468

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Considered Spatial-Unit:  None
Invalid dates within this fold: 241

Init Dataset: 'torch.Size([2294, 20]) with 45880 Total nb of elements and 0 Nan values
nb CRITER_3lanes invalid dates:  241
vision_input_type POIs
vision_model_name VariableSelectionNetwork
Init U/Utarget size: torch.Size([1467, 20, 7])/torch.Size([1467, 20, 1]) Train/Valid/Test 530 469 467

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
number of Parame

In [3]:
from plotting.TS_analysis import plot_TS


stations_to_plot = [893,67,999,1176] #65,291,1176] # [292,64,66,1000] #[893,67,999,65,291,1176]
df_pred1,df_true,index_df = get_TS_prediction(trainer,ds,stations_to_plot=stations_to_plot,training_mode='test',name= 'CRITER')
df_pred2,df_true,index_df = get_TS_prediction(trainer2,ds2,stations_to_plot=stations_to_plot,training_mode='test',name= 'CRITER_netmob_waze')
df_pred_all_models = pd.concat([df_pred1,df_pred2,df_true],axis=1)
df_pred_all_models.index = index_df
plot_TS(df_pred_all_models,width=1200,height=400,bool_show=True,title='Comparison of Prediction Quality')

ValueError: 893 is not in list